# Iterated Prisoner's Dilemma On A Network

## Imports and Config

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 500

In [ ]:
from Prison import ActionStrategy, ImitationStrategy, FermiStrategy, ReinforcementLearningStrategy, TitForTatStrategy
from Prison import Agent, Network, NetworkSimulation
from Prison import experiment
from Prison import payoff_matrices

In [ ]:
def calculate_percolation_metrics(simulation):
    """
    Calculates the size of the largest cooperative cluster (S_max)
    and the susceptibility (chi).
    """
    # 1. Identify Cooperator Nodes
    state = simulation._get_state() # Returns dict {node_id: 0(C) or 1(D)}
    coop_nodes = [n for n, action in state.items() if action == 0] # 0 is C in state01 logic
    
    N = simulation.graph.number_of_nodes()
    
    # If no cooperators, metrics are 0
    if not coop_nodes:
        return 0.0, 0.0

    # 2. Build Cooperative Subgraph
    # We use the subgraph method to find connected components of cooperators
    G_coop = simulation.graph.subgraph(coop_nodes)
    components = list(nx.connected_components(G_coop))
    
    # 3. Get Sizes
    sizes = sorted([len(c) for c in components], reverse=True)
    s_max_count = sizes[0]
    
    # Normalized S_max (Order Parameter)
    S_max = s_max_count / N 
    
    # 4. Calculate Susceptibility (Chi)
    # Filter out the largest component (s_max)
    # If there is only one component, the sum over s != s_max is empty -> chi = 0
    other_sizes = sizes[1:]
    
    if not other_sizes:
        chi = 0.0
    else:
        # Numerator: sum(s^2 * n_s) excluding s_max
        # Since 'other_sizes' is a list of individual cluster sizes, 
        # sum(s^2) over this list is equivalent to sum(s^2 * n_s)
        numerator = sum(s**2 for s in other_sizes)
        
        # Denominator: sum(s * n_s) excluding s_max
        denominator = sum(other_sizes)
        
        chi = numerator / denominator if denominator > 0 else 0.0
        
    return S_max, chi



In [ ]:
# --- Configuration ---
N_SIDE = 30           # 30x30 grid = 900 agents
STEPS = 1500          # Steps to reach equilibrium
TRIALS = 5            # Averaging over random seeds (increase for smoother curves)
BETAS = np.linspace(0.0, 1.0, 20) # Sweep parameter

results = []

print(f"Running percolation sweep on {N_SIDE}x{N_SIDE} grid over {len(BETAS)} beta values...")



In [ ]:
# --- Experiment Loop ---
for beta in tqdm(BETAS):
    # Determine b and c based on beta (b/c = beta * kbar)
    # For a grid, kbar = 4
    kbar = 4
    c = 1.0
    b = beta * kbar * c
    
    # Create payoff matrix manually or use the helper class
    # Using the helper class logic directly for clarity:
    pm = {
        ("C", "C"): (b - c, b - c),
        ("C", "D"): (-c, b),
        ("D", "C"): (b, -c),
        ("D", "D"): (0, 0),
    }

    s_max_list = []
    chi_list = []
    
    for _ in range(TRIALS):
        # Initialize Simulation
        sim = NetworkSimulation(
            kind="grid",
            n=N_SIDE*N_SIDE,
            payoff_matrix=pm,
            strategy=FermiStrategy,
            strategy_kwargs={'temperature': 0.1}, # Low temp for sharper transitions
            store_history=False,    # Optimization
            store_snapshots=False   # Optimization
        )
        
        # Run to steady state
        sim.run_until_attractor(max_steps=STEPS, check_every=50, store_cycle_states=False)
        
        # Measure
        s_max, chi = calculate_percolation_metrics(sim)
        s_max_list.append(s_max)
        chi_list.append(chi)
        
    # Aggregate results
    results.append({
        "beta": beta,
        "b_over_c": b/c,
        "S_max_mean": np.mean(s_max_list),
        "S_max_std": np.std(s_max_list),
        "chi_mean": np.mean(chi_list),
        "chi_std": np.std(chi_list)
    })

df_res = pd.DataFrame(results)

# --- Visualization ---
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Benefit-to-Cost Ratio ($b/c$)')
ax1.set_ylabel('Giant Component size ($S_{max}$)', color=color, fontweight='bold')
ax1.errorbar(df_res['b_over_c'], df_res['S_max_mean'], yerr=df_res['S_max_std'], 
             color=color, marker='o', label='$S_{max}$')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

# Instantiate a second axes that shares the same x-axis
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Susceptibility ($\chi$)', color=color, fontweight='bold')  
ax2.errorbar(df_res['b_over_c'], df_res['chi_mean'], yerr=df_res['chi_std'], 
             color=color, marker='x', linestyle='--', label='$\chi$')
ax2.tick_params(axis='y', labelcolor=color)

plt.title(f"Cooperative Percolation Transition (Grid {N_SIDE}x{N_SIDE})")
plt.tight_layout()
plt.show()